## **1. Import Librairies:**

In [1]:
import pandas as pd
import requests
from datetime import datetime
from datetime import timedelta
import pickle

## **2. Load Datasets:**

In [2]:
timetables = pd.read_csv('timetables.csv', sep="\t")
routes = pd.read_csv('data_sncf/routes.csv', sep=",")

### **a. Timetable:**

In [3]:
timetables

,trip_id,trajet,duree
0,OCESN003100F140147152,Gare de Le Havre - Gare de Paris-St-Lazare,138
1,OCESN003190F040047309,Gare de Dieppe - Gare de Paris-St-Lazare,145
2,OCESN003198F030037315,Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...,97
3,OCESN003300F030037323,Gare de Cherbourg - Gare de Paris-St-Lazare,194
4,OCESN003313F380387526,Gare de Caen - Gare de Paris-St-Lazare,149
...,...,...,...
1570,OCESN895822F0500552575,Gare de Belfort-Ville - Gare de Lyon-Perrache,244
1571,OCESN895830F0200252600,Gare de Lons-le-Saunier - Gare de Lyon-Perrache,103
1572,OCESN895880F0500552634,Gare de Belfort-Ville - Gare de Lons-le-Saunier,144
1573,OCESN895940F0200252654,Gare de Besançon-Viotte - Gare de Lons-le-Saunier,89


### **b. Routes:** 

In [4]:
routes

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,OCE1506035,OCESN,NaN,Paris-Vernon-Rouen-Le Havre,NaN,2,NaN,NaN,NaN
1,OCE1506037,OCESN,NaN,Paris-Caen-Cherbourg/Trouville,NaN,2,NaN,NaN,NaN
2,OCE1506036,OCESN,NaN,Paris-Granville,NaN,2,NaN,NaN,NaN
3,OCE1209987,OCESN,NaN,Paris-Bourges-Montluçon,NaN,2,NaN,NaN,NaN
4,OCE1417992,OCESN,NaN,Paris - Nevers,NaN,2,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
707,OCESN-87743005-87718833,OCESN,NaN,Bourg-en-Bresse - Mouchard,NaN,2,NaN,NaN,NaN
708,OCEZW-87576025-87576215,OCESN,NaN,Romorantin-Blanc-Argent - Valençay,NaN,2,NaN,NaN,NaN
709,OCEZW-87576025-87543165,OCESN,NaN,Romorantin-Blanc-Argent - Salbris,NaN,2,NaN,NaN,NaN
710,OCEZW-87543165-87576215,OCESN,NaN,Salbris - Valençay,NaN,2,NaN,NaN,NaN


### **c. Stops:**

In [5]:
f = open("cities.pkl", "rb")
cities = pickle.load(f)
f.close()

if not cities:
    stops = pd.read_csv('data_sncf/stops.csv', sep=",")
    stops = stops[stops['stop_id'].str.contains('StopPoint:OCETrain')]
    stops = stops.set_index('stop_id').T.to_dict()
    cities = {}

    for city in list(stops.items()):
        cities.update({
            city[0]: {
                "stop_name": city[1]["stop_name"],
                "coord": [city[1]["stop_lat"], city[1]["stop_lon"]],
        }})

len(list(cities.items()))

2447

In [6]:
# if not cities:
cities_file = open("cities.pkl", "wb")

pickle.dump(cities, cities_file)

cities_file.close()

In [7]:
cities["StopPoint:OCETrain TER-87381509"]

{'stop_name': 'Gare de Mantes-la-Jolie', 'coord': [48.98968735, 1.70329418]}

In [8]:
f = open("trips.pkl", "rb")
trips_tmp = pickle.load(f)
f.close()

if trips_tmp is None:
    stop_times = pd.read_csv('data_sncf/stop_times.csv', sep=",")
    trips = pd.read_csv('data_sncf/trips.csv', sep=",")

    stop_times = stop_times[stop_times['stop_id'].str.contains('StopPoint:OCETrain')]


    trips = trips.drop(labels=["service_id", "block_id", "shape_id", "trip_headsign"], axis=1)
    print("trips normlaization")

    trips = trips.set_index('trip_id').T.to_dict()
    print("trips to dict")

    trips_tmp = {}
    for trip in list(trips.items()):
        trips_tmp.update({trip[0]:{"nodes": []}})
        selected_stop_times = stop_times.loc[stop_times['trip_id'] == trip[0]]
        for trip_tmp in selected_stop_times.iterrows():
            trips_tmp[trip[0]]["nodes"].append({"trip_id": trip[0], "stop_id": trip_tmp[1]["stop_id"],"arrival_time": trip_tmp[1]["arrival_time"]})


In [9]:
trips_file = open("trips.pkl", "wb")

pickle.dump(trips_tmp, trips_file)

trips_file.close()

In [10]:
s1 = '23:05:00'
s2 = '25:10:00'
s3 = '26:10:00'

def get_duration_node_to_node(origin, dest):
    start_minute = int(origin.split(":")[1])
    start_hour = int(origin.split(":")[0])

    arrival_minute = int(dest.split(":")[1])
    arrival_hour = int(dest.split(":")[0])

    start_delta = timedelta(
        seconds=0,
        minutes=start_minute,
        hours=start_hour,
     )

    arrival_delta = timedelta(
        seconds=0,
        minutes=arrival_minute,
        hours=arrival_hour,
     )

    return arrival_delta - start_delta

first_date = get_duration_node_to_node(s1,s2)
second_date = get_duration_node_to_node(s1,s3)

print(first_date)
print(second_date)

print(first_date < second_date)

2:05:00
3:05:00
True


## **3. Normalization:**

### **a. routes:**

In [11]:
# routes.drop(columns=['route_short_name', 'route_desc', 'route_url', 'route_color', 'route_text_color'])

## **4. Train Stations Structure:**

### **c. Alternative data type:**

In [12]:
f = open("graph.pkl", "rb")
graph = pickle.load(f)
f.close()

if graph:
    routes_graph = {}
    for route in list(trips_tmp.items()):
        for i in range (0, len(route[1]["nodes"])):
            city_id = route[1]["nodes"][i]["stop_id"]
            city = cities[city_id]
            if city_id not in routes_graph:
                routes_graph.update({city_id: []})
                if i != 0:
                    routes_graph[city_id].append({route[1]["nodes"][i - 1]["stop_id"]: get_duration_node_to_node(route[1]["nodes"][i - 1]["arrival_time"], route[1]["nodes"][i]["arrival_time"])})
                if i != len(route[1]["nodes"]) - 1:
                    routes_graph[city_id].append({route[1]["nodes"][i + 1]["stop_id"]: get_duration_node_to_node(route[1]["nodes"][i]["arrival_time"], route[1]["nodes"][i + 1]["arrival_time"])})
            elif route[1]["nodes"][i - 1]["stop_id"] not in routes_graph[city_id] and i != 0:
                routes_graph[city_id].append({route[1]["nodes"][i - 1]["stop_id"]: get_duration_node_to_node(route[1]["nodes"][i - 1]["arrival_time"], route[1]["nodes"][i]["arrival_time"])})
            elif i < len(route[1]["nodes"]) - 1 and route[1]["nodes"][i + 1]["stop_id"] not in routes_graph[city_id]:
                routes_graph[city_id].append({route[1]["nodes"][i + 1]["stop_id"]: get_duration_node_to_node(route[1]["nodes"][i]["arrival_time"], route[1]["nodes"][i + 1]["arrival_time"])})


In [13]:
graph_file = open("graph.pkl", "wb")

pickle.dump(routes_graph, graph_file)

graph_file.close()

## **6. Graph Exploration:** 

In [14]:
def BFS_SP(graph, start, goal):
    explored = []
    queue = [[start]]
     
    # If the desired node is
    # reached
    if start == goal:
        print("Same Node")
        return
    
    valide_routes = []
    while queue:
        path = queue.pop(0)
        node = path[-1]
        
        if node not in explored and node != goal:
            node_id = list(node.keys())[0]
            neighbours = graph[node_id]
            duration = timedelta(hours=0)
            for neighbour in neighbours:
                if list(neighbour)[0] in cities:
                    new_path = list(path)
                    new_path.append(neighbour)
                    queue.append(new_path)
                    duration = duration + neighbour[list(neighbour)[0]]
                    if list(neighbour)[0] == list(goal)[0]:
                        if len(valide_routes) > 25:
                            print("valides routes found")
                            return valide_routes
                        print("new route found")
                        valide_routes.append({"route": new_path, "duration": duration})
            explored.append(node)
    return valide_routes

In [15]:
def display_routes(routes):
    if len(routes) > 0:
        less_node = len(routes[0]["route"])
        less_node_nbr = routes[0]

        shortest_duration = routes[0]["duration"]
        shortest_route = routes[0]
        nbr_routes = len(routes)

        for route in routes:
        #     print("route")
        #     print(route)
        #     print("->".join(cities[route["route"]]))
            if route["duration"] < shortest_duration:
                shortest_duration = route["duration"]
                shortest_route = route
            if len(route["route"]) < less_node:
                less_node = len(route)
                less_node_nbr = route
        print("routes: ", nbr_routes)
        print("Shortest path :")
        print(shortest_duration)
        for city in shortest_route["route"]:
            print(cities[list(city)[0]]["stop_name"])
        #         print(cities[list(city)[0]]["stop_id"])
            print("|")
        print("==========================")
        print("==========================")

        print("least nbr of node:", less_node)
        print(shortest_duration)
        for city in less_node_nbr["route"]:
            print(cities[list(city)[0]]["stop_name"])
        #         print(cities[list(city)[0]]["stop_id"])
            print("|")
    else:
        print("No route")

In [16]:
origin = {"StopPoint:OCETrain TER-87384008": timedelta(hours=0)}
dest = {"StopPoint:OCETrain TER-87411017": timedelta(hours=0)}

print(cities[list(origin)[0]]["stop_name"])
print(cities[list(dest)[0]]["stop_name"])

Gare de Paris-St-Lazare
Gare de Rouen-Rive-Droite


In [17]:
routes = BFS_SP(routes_graph, origin, dest)

new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
valides routes found


In [18]:
display_routes(routes)

routes:  26
Shortest path :
1:18:00
Gare de Paris-St-Lazare
|
Gare de Rouen-Rive-Droite
|
least nbr of node: 2
1:18:00
Gare de Paris-St-Lazare
|
Gare de Rouen-Rive-Droite
|


In [19]:
origin = {"StopPoint:OCETrain TER-87384008": timedelta(hours=0)}
dest = {"StopPoint:OCETrain TER-87611004": timedelta(hours=0)}

print(cities[list(origin)[0]]["stop_name"])
print(cities[list(dest)[0]]["stop_name"])

Gare de Paris-St-Lazare
Gare de Toulouse-Matabiau


In [20]:
routes = BFS_SP(routes_graph, origin, dest)

new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
new route found
valides routes found


In [21]:
display_routes(routes)

routes:  26
Shortest path :
2:33:00
Gare de Paris-St-Lazare
|
Gare de Caen
|
Gare de Argentan
|
Gare de Sées
|
Gare de Alençon
|
Gare de Le Mans
|
Gare de Château-du-Loir
|
Gare de Tours
|
Gare de Les Ormes-sur-Vienne
|
Gare de Dangé
|
Gare de Chatellerault
|
Gare de Futuroscope
|
Gare de Poitiers
|
Gare de Lussac-les-Châteaux
|
Gare de Montmorillon
|
Gare de Le Dorat
|
Gare de Bellac
|
Gare de Nantiat
|
Gare de Limoges-Bénédictins
|
Gare de Uzerche
|
Gare de Allassac
|
Gare de Brive-la-Gaillarde
|
Gare de Souillac
|
Gare de Gourdon
|
Gare de Cahors
|
Gare de Lalbenque-Fontanes
|
Gare de Caussade (T.-et-G.)
|
Gare de Albias
|
Gare de Montauban-Ville-Bourbon
|
Gare de Castelnau-d'Estrétefond
|
Gare de Toulouse-Matabiau
|
least nbr of node: 31
2:33:00
Gare de Paris-St-Lazare
|
Gare de Caen
|
Gare de Argentan
|
Gare de Sées
|
Gare de Alençon
|
Gare de Le Mans
|
Gare de Château-du-Loir
|
Gare de Tours
|
Gare de Les Ormes-sur-Vienne
|
Gare de Dangé
|
Gare de Chatellerault
|
Gare de Futurosc